# Generating Shakespearean Text

## Creating the Training Dataset

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

c:\users\ayanle\appdata\local\programs\python\python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt", url,
                               cache_dir = '.')

In [3]:
with open(filepath) as f:
    text = f.read()

We have now downloaded the dataset. Now we need to encode every character as an integer. To do this we can use Keras Tokenizer class which maps every character in the text to a unique character id from 1 to the number of distinct characters.

In [4]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([text])

Here set char_level = True to get character level encoding instead of the default word-level encoding.

In [5]:
tokenizer.texts_to_sequences(['last'])

[[12, 5, 8, 3]]

In [6]:
tokenizer.sequences_to_texts([[12, 5, 8, 3]])

['l a s t']

In [7]:
distinct_ids = len(tokenizer.word_index) # number of distinct characters
distinct_ids

39

In [8]:
[encoded] = np.array(tokenizer.texts_to_sequences([text])) - 1

In [9]:
len(encoded) # total number of characters

1115394

## How to Split a Sequential Dataset

It is very important to avoid any overlap between the training set, the validation set, and the test set. For example we use the first 90% for training the next 5% for validation and the final 5% for the test set.

In [10]:
train_size = len(encoded) * 90 // 100

In [11]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

## Chopping the Sequential Dataset into Multiple Windows

In [12]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True) # creates a datset of windows and each window is a dataset

In [13]:
dataset = dataset.flat_map(lambda window: window.batch(window_length)) # creates a dataset of tensors of length window_length.
                                                                       # This essentially flattens the nested dataset above.

In [14]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:])) # separate inputs from target

In [15]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=distinct_ids), Y_batch))

In [16]:
dataset = dataset.prefetch(1)

# Building and Training the RNN Model

Since we want to predict the next character based on the previous 100 characters, we can use an RNN with GRU layers of 128 units each. The output is a time distributed dense layer with 39 units for each character and a softmax activation function

In [17]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, distinct_ids]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(distinct_ids, activation='softmax'))
    
    
])

In [18]:
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam')

In [ ]:
history = model.fit(dataset, epochs=20)

Epoch 1/20
  15385/Unknown - 314s 19ms/step - loss: 0.9869

In [ ]:
def preprocess(texts):
 X = np.array(tokenizer.texts_to_sequences(texts)) - 1
 return tf.one_hot(X, distinct_ids)

In [ ]:
X_new = preprocess(["how are yo"])
Y_pred = np.argmax(model.predict(X_new), axis=1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

In [ ]:
Y_pred[0]